In [17]:
import os
import sys
from src.preprocessing.load_data import *
from src.preprocessing.process_softchirp_features import find_outliers, choose_features
from src.data_representations.softchirp_features import compute_all_features
from src.data_representations.process_traces import clear_trace
from src.preprocessing.filters import filter_recs_metadata, filter_sounds_metadata_extended
from src.preprocessing.process_sounds_metadata import extend_sounds_metadata, compute_epochs
from src.visualization.plot_softchirps import plot
from src.util import delete_indexes
import src.visualization.plot_softchirps
from notebooks.analysis.plotting_config import ORDER, COLOR, EPOCHS # change plotting_config file to change colors
from src.util import date_to_datetime
import numpy as np
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from collections import defaultdict
import matplotlib
import matplotlib.cm as cm
%matplotlib inline

In [18]:
PATH_TO_PROJECT = './'

In [19]:
cols = ['baratheon', 'dothrakib', 'targaryen', 'martell', 'stark']

dates_limits = {'baratheon' : ('24-03-19', '20-06-20'),
                'dothrakib': ('24-03-19', '03-11-19'),
                'targaryen': ('24-03-19', '03-11-19'),
                'martell': ('24-03-19', '03-12-19'),
                'stark': ('01-04-17', '30-06-18')}

In [35]:
recs_metadata_new[recs_metadata_new['colony'] == 'stark'].date.unique()

array(['26-09-17', '01-11-17', '19-06-18', '01-12-17', '05-10-17',
       '21-08-17', '31-03-18', '07-07-17', '15-05-18', '27-11-17',
       '06-02-18', '05-09-17', '23-12-17'], dtype=object)

In [20]:
DATA_FOLDER_NAME = '/data/'
path_to_info = PATH_TO_PROJECT + DATA_FOLDER_NAME + '/rat_info.csv'
path_to_recordings_metadata = PATH_TO_PROJECT + DATA_FOLDER_NAME
recordings_metadata_name = 'recordings_metadata.csv'

info = load_info_file(path_to_info)
recs_metadata = pd.read_csv(path_to_recordings_metadata + recordings_metadata_name, dtype={'ratids': str})
recs_metadata = recs_metadata[recs_metadata['colony'].isin(cols)].reset_index(drop=True)

In [21]:
ratids_are_in_info = np.ones(recs_metadata.shape[0], dtype='bool')
single_ratids = recs_metadata['ratids'].apply(lambda x: len(x) <= 5)
unique_ratids = recs_metadata[single_ratids]['ratids'].unique()
rids_delete = set()
for rid in unique_ratids:
    if int(rid) not in info['ID'].values:
        print(rid)
        rids_delete.add(rid)
        
mask_rids_delete = recs_metadata['ratids'].isin(rids_delete)

9466
4053
1407
3979
4043
3981
3985
2159
3984
2041
4032
4020
4012
3900
4018
4202
3914
4006
4065
4063
4023
3902


In [22]:
recs_metadata_new = pd.DataFrame(np.copy(recs_metadata), columns=recs_metadata.columns)
mask = np.zeros(recs_metadata_new.shape[0], dtype='bool')
ratids_are_in_info = np.ones_like(mask, dtype='bool')

for c in cols:
    mask_c = recs_metadata.colony == c
    
    dates = dates_limits[c]
    dates_transformed = recs_metadata_new['date'].apply(date_to_datetime)
    d_min, d_max = date_to_datetime(dates[0]), date_to_datetime(dates[1])
    mask_d = dates_transformed.apply(lambda x: d_min <= x <= d_max)
    if c == 'stark':
        mask_ratids = np.ones_like(mask, dtype='bool')
    else:
        mask_ratids = recs_metadata['ratids'].apply(lambda x: len(x) <= 5)
    mask_c = mask_c * mask_d * mask_ratids
    print(c, np.sum(mask_c))
    mask[mask_c] = True
    
mask[mask_rids_delete] = False
mask_delete = np.invert(mask)

recs_to_delete = recs_metadata[mask_delete].reset_index(drop=True)
recs_metadata_new = recs_metadata[mask].reset_index(drop=True)

baratheon 183
dothrakib 25
targaryen 77
martell 94
stark 304


In [23]:
mask_ratinfo = np.zeros(len(info), dtype='bool')
for rid in recs_metadata_new['ratids'].unique():
    rat_inds = np.where(info['ID'] == int(rid))[0]
    if len(rat_inds) == 0:
        print(rid, 'no ')
    else:
        mask_ratinfo[rat_inds] = True
info_new = info[mask_ratinfo].reset_index(drop=True)

8406_9453 no 
9478_8406 no 
8304_9462 no 
8304_9660 no 
9660_8314 no 
9474_8322 no 
9551_9597 no 
8406_9551 no 
8322_9453 no 
8318_8388 no 
9597_9551 no 
9462_8406 no 
1556_9597 no 
9478_1556 no 
8406_9478 no 
8318_9453 no 
8529_9478 no 
9597_8304 no 
9453_8529 no 
9660_1549 no 
8476_8529 no 
9551_8314 no 
8322_9660 no 
1545_1549 no 
9453_9551 no 
9453_8322 no 
8529_9474 no 
9597_1556 no 
9551_8529 no 
8314_9551 no 
8318_9660 no 
8529_8476 no 
1549_9597 no 
8318_8406 no 
9478_9597 no 
9660_9597 no 
9597_8318 no 
9597_8406 no 
8476_9660 no 
8406_1545 no 
9478_9453 no 
9551_8406 no 
1549_9478 no 
8406_9474 no 
9478_9660 no 
8476_9597 no 
9462_9478 no 
9551_9462 no 
9453_9474 no 
9474_9551 no 
9462_9453 no 
8322_8529 no 
9597_8314 no 
8304_8476 no 
8318_9478 no 
8304_8388 no 
8388_9660 no 
8476_9474 no 
9478_9462 no 
8318_9474 no 
1556_8406 no 
8406_9660 no 
8318_9551 no 
9551_8322 no 
9551_9660 no 
9453_8406 no 
9660_8304 no 
9660_9551 no 
8529_1549 no 
9453_9660 no 
8314_8322 no 
8529_9

In [25]:
for rec_name, rec_path, st in recs_to_delete[['name', 'path', 'processing stage']].values:
    traces_path = rec_path.replace('recordings', 'traces')
    rec_file = rec_path + rec_name
    txt_file = rec_path + rec_name.replace('npy', 'txt')
    rec_traces = [t for t in os.listdir(traces_path)  if (rec_name[:-3] in t) and 'trace' in t]
    try:
        os.remove(rec_file)
        os.remove(txt_file)
        for tr in rec_traces:
            os.remove(traces_path + tr)
    except:
        pass


In [32]:
info_new.to_csv(path_to_info, index=None)
recs_metadata_new.to_csv(path_to_recordings_metadata + recordings_metadata_name, index=None)